# Link Prediction
In this file we will give you a simple example to show how to use AutoGL to do link prediction.

## Import libraries
First, you should import some libraries and you can set the random seed before you train the model.

In [ ]:
from autogl.datasets import build_dataset_from_name
from autogl.solver import AutoLinkPredictor
from autogl.solver.utils import set_seed
import argparse
from autogl.backend import DependentBackend
from autogl.datasets.utils import split_edges
from autogl.module.train.evaluation import Auc
import yaml
import random
import torch
import numpy as np
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
set_seed(202106)

## Config & Dataset
Then, you can load dataset and graph nas methods from configs

In [ ]:
def parse(args=None):
    parser = ArgumentParser("auto link prediction", formatter_class=ArgumentDefaultsHelpFormatter)
    parser.add_argument('--config', type=str, default="../../configs/lp_gcn_benchmark.yml")
    parser.add_argument('--dataset', choices=['cora', 'citeseer', 'pubmed'], default='cora', type=str)
    parser.add_argument("--hpo", type=str, default="tpe", help="hpo methods")
    parser.add_argument("--max_eval", type=int, default=50, help="max hpo evaluation times")
    parser.add_argument("--seed", type=int, default=0, help="random seed")
    parser.add_argument("--device", default=0, type=int, help="GPU device")
    args = parser.parse_args(args)
    return args

args = parse('')

dataset = build_dataset_from_name(args.dataset)
dataset = split_edges(dataset, 0.8, 0.05) # split the edges for dataset
if DependentBackend.is_dgl(): # add self-loop
    import dgl
    # add self loop to 0
    data = list(dataset[0])
    data[0] = dgl.add_self_loop(data[0])
    dataset = [data]
configs = yaml.load(open(args.config, "r").read(), Loader=yaml.FullLoader)
configs["hpo"]["name"] = args.hpo
configs["hpo"]["max_evals"] = args.max_eval
solver = AutoLinkPredictor.from_config(configs)

## Run 

In [ ]:
solver.fit(dataset, time_limit=3600, evaluation_method=[Auc], seed=args.seed)
solver.get_leaderboard().show()
auc = solver.evaluate(metric="auc")
print("test auc: {:.4f}".format(auc))